In [1]:
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import nibabel
import nibabel.processing
import os

from scipy import ndimage

from matplotlib import pyplot
import os
from skimage.filters import threshold_otsu
import cc3d
from tensorflow.keras import Input, Model
from scipy.ndimage import zoom
from matplotlib import animation, rc
from IPython import display 

In [2]:
#Borrowed from the 3D preprocessing

def normalize(volume):
    """Normalize the volume"""
    min = volume.min()
    max = volume.max()
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume

def pp(original_image):
    
    input_img=input_img = nibabel.load("{0}".format(original_image))
    resampled_img = nibabel.processing.conform(input_img, out_shape=(100,100,90), voxel_size=(2.0, 2.0, 2.0))
    
    img = resampled_img.get_fdata()
    
    thresh = threshold_otsu(img)
    bw_img1 = np.copy(img)
    bw_img1[bw_img1 < thresh] = 0
    bw_img1[bw_img1 >= thresh] = 255
    
    input_CCA=bw_img1.astype('int32')
    connectivity = 6
    labels_out, N = cc3d.connected_components(input_CCA, return_N=True)
    
    def mask_largest_label (labels_out, N):
        #print("This function returns the largest blob of a CCA processed image as a binary mask")
        #print("")
        def separate_labels(label_ID, label_matrix):
            mask=1*(label_matrix == label_ID)
            return mask
        labellist=[]
        for j in range(1, N+1):
            a=separate_labels(j, labels_out)
            labellist.append(a)
        #print("The image has {0} labels".format(len(labellist)))
        z=labellist[0]
        #print("The shape of the labels is: {0}".format(z.shape))
        sizelist=[]
        for counter,element in enumerate (labellist):
            a=labellist[counter].sum()
            sizelist.append(a)
        #print("Label sizes: {0}".format(sizelist))
        sizelist=np.asarray(sizelist)
        a=sizelist.argmax()
        #print("The largest label index is: {0}".format(a))
        mask=labellist[a]
        #print("The largest label is now a binary mask with shape {0}, size {1}, max value {2} and min value {3}".format((mask.shape),(mask.sum()),(mask.max()),(mask.min())))
        return mask

    mask=mask_largest_label(labels_out, N)
    
    pimg=np.multiply(img,mask)
    
    return pimg

def process(path):
    """Read and normalize volume"""
    # Read and pp scan
    volume = pp(path)
    # Normalize
    volume = normalize(volume)
    #Prepare for NN input
    volume = np.expand_dims(volume, axis=0)
    volume = np.expand_dims(volume, axis=4)
    return volume

In [3]:
volume_size = (100, 100, 90, 1)
last_conv_layer_name = 'conv3d_4'

In [4]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """Generate class activation heatmap"""
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = Model(
        [model.inputs], [model.get_layer(
            last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel (equivalent to global average pooling)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2, 3))

    # We multiply each channel in the feature map array
    # by 'how important this channel is' with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    # Notice that we clip the heatmap values, which is equivalent to applying ReLU
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [5]:
def get_resized_heatmap(heatmap, shape):
    """Resize heatmap to shape"""
    # Rescale heatmap to a range 0-255
    upscaled_heatmap = np.uint8(255 * heatmap)

    upscaled_heatmap = zoom(
        upscaled_heatmap,
        (
            shape[0] / upscaled_heatmap.shape[0],
            shape[1] / upscaled_heatmap.shape[1],
            shape[2] / upscaled_heatmap.shape[2],
        ),
    )

    return upscaled_heatmap

# Loop fold 1

In [6]:
modeldir="/local_mount/space/celer/1/users/notebooks/moises/saved_models/newconv_originalsmallcnn0/epoch16/"

In [7]:
model=tf.keras.models.load_model(modeldir)

In [8]:
dire="/local_mount/space/celer/1/users/notebooks/moises/pdata/newdata/kfold2/fold1/pMCI"

In [9]:
for i in os.listdir(dire):
                print("a")
                volume=process(os.path.join(dire,'{}'.format(i)))
                prediction=model.predict(volume)
                print("a")
                score=prediction[0][0]
                print(i)
                print(score)
                model.layers[-1].activation = None
                heatmap = make_gradcam_heatmap(volume, model, last_conv_layer_name)
                resized_heatmap = get_resized_heatmap(heatmap, volume_size)
                volume=np.squeeze(volume)
                print("a")
                
                input_img= nibabel.load(os.path.join(dire,'{}'.format(i)))
                aff=input_img.affine
                
                saved_img = nibabel.Nifti1Image(volume, aff)
                nibabel.save(saved_img, '/local_mount/space/celer/1/users/notebooks/moises/processed_nifty/fold1/pMCI/{}.nii'.format(i))
                print("a")
                heaty=normalize(resized_heatmap)
                saved_heatmap = nibabel.Nifti1Image(heaty, aff)
                nibabel.save(saved_heatmap, '/local_mount/space/celer/1/users/notebooks/moises/heatmaps/fold1/pMCI/{}.nii'.format(i))
                

a
a
128_S_0135_I192851.nii
0.03223298
a
a
a
a
027_S_0461_I26955.nii
0.88741285
a
a
a
a
005_S_0448_I225803.nii
0.6360459
a
a
a
a
007_S_0698_I23430.nii
0.58614427
a
a
a
a
021_S_0231_I12047.nii
0.9908441
a
a
a
a
009_S_1030_I29585.nii
0.31360927
a
a
a
a
006_S_1130_I221823.nii
0.7984454
a
a
a
a
007_S_0344_I13813.nii
0.79175544
a
a
a
a
100_S_1154_I71927.nii
0.66911596
a
a
a
a
137_S_0973_I33206.nii
0.054149754
a
a
a
a
033_S_0723_I20277.nii
0.5046968
a
a
a
a
035_S_0204_I12931.nii
0.22358294
a
a
a
a
116_S_1315_I48322.nii
0.3721628
a
a
a
a
018_S_0155_I12924.nii
0.8204797
a
a
a
a
057_S_1217_I40094.nii
0.916516
a
a
a
a
013_S_0325_I17295.nii
0.6379134
a
a
a
a
141_S_1004_I282482.nii
0.8400152
a
a
a
a
116_S_1243_I41792.nii
0.36853182
a
a
a
a
005_S_0222_I12074.nii
0.38040748
a
a
a
a
136_S_0873_I204298.nii
0.9375163
a
a
a
a
057_S_1265_I39726.nii
0.6379433
a
a


In [10]:
dire="/local_mount/space/celer/1/users/notebooks/moises/pdata/newdata/kfold2/fold1/sMCI"

In [11]:
for i in os.listdir(dire):
                volume=process(os.path.join(dire,'{}'.format(i)))
                prediction=model.predict(volume)
                score=prediction[0][0]
                print(i)
                print(score)
                model.layers[-1].activation = None
                heatmap = make_gradcam_heatmap(volume, model, last_conv_layer_name)
                resized_heatmap = get_resized_heatmap(heatmap, volume_size)
                volume=np.squeeze(volume)
                
                input_img= nibabel.load(os.path.join(dire,'{}'.format(i)))
                aff=input_img.affine
                
                saved_img = nibabel.Nifti1Image(volume, aff)
                nibabel.save(saved_img, '/local_mount/space/celer/1/users/notebooks/moises/processed_nifty/fold1/sMCI/{}.nii'.format(i))
                
                heaty=normalize(resized_heatmap)
                saved_heatmap = nibabel.Nifti1Image(heaty, aff)
                nibabel.save(saved_heatmap, '/local_mount/space/celer/1/users/notebooks/moises/heatmaps/fold1/sMCI/{}.nii'.format(i))
                

168_S_6180_I983061.nii
0.32651
168_S_6467_I1034450.nii
0.28176263
019_S_6315_I997787.nii
0.77742094
041_S_1411_I76843.nii
0.27783972
070_S_6236_I972920.nii
0.087942734
168_S_6860_I1325690.nii
0.19814207
005_S_0546_I17723.nii
0.13432607
041_S_0407_I16606.nii
0.14203572
031_S_0554_I16828.nii
0.7178184
027_S_1045_I209627.nii
0.38391608
037_S_0150_I257571.nii
0.80670536
016_S_6789_I1243165.nii
0.095670916
109_S_0950_I29821.nii
0.3306128
012_S_6503_I1074989.nii
0.03591629
041_S_0598_I20051.nii
0.4177672
003_S_6479_I1065559.nii
0.193752
037_S_6083_I938655.nii
0.008185278
153_S_6633_I1090964.nii
0.1325295
135_S_6544_I1050687.nii
0.012596545
168_S_6591_I1158154.nii
0.28598982
027_S_6002_I829731.nii
0.06845065
009_S_1199_I37964.nii
0.34859905
035_S_0033_I16117.nii
0.5645479
012_S_1175_I36789.nii
0.09756909
129_S_6852_I1296763.nii
0.077976026
116_S_6428_I1024238.nii
0.0071057947
073_S_1357_I57533.nii
0.18741056
027_S_6034_I908623.nii
0.0405575
116_S_6775_I1234267.nii
0.17047112
006_S_6682_I11575

# Loop fold 2

In [12]:
modeldir="/local_mount/space/celer/1/users/notebooks/moises/saved_models/newconv_originalsmallcnn1/epoch19"

In [13]:
model=tf.keras.models.load_model(modeldir)

In [14]:
dire="/local_mount/space/celer/1/users/notebooks/moises/pdata/newdata/kfold2/fold2/pMCI"

In [15]:
for i in os.listdir(dire):
                volume=process(os.path.join(dire,'{}'.format(i)))
                prediction=model.predict(volume)
                score=prediction[0][0]
                print(i)
                print(score)
                model.layers[-1].activation = None
                heatmap = make_gradcam_heatmap(volume, model, last_conv_layer_name)
                resized_heatmap = get_resized_heatmap(heatmap, volume_size)
                volume=np.squeeze(volume)
                
                input_img= nibabel.load(os.path.join(dire,'{}'.format(i)))
                aff=input_img.affine
                
                saved_img = nibabel.Nifti1Image(volume, aff)
                nibabel.save(saved_img, '/local_mount/space/celer/1/users/notebooks/moises/processed_nifty/fold2/pMCI/{}.nii'.format(i))
                
                heaty=normalize(resized_heatmap)
                saved_heatmap = nibabel.Nifti1Image(heaty, aff)
                nibabel.save(saved_heatmap, '/local_mount/space/celer/1/users/notebooks/moises/heatmaps/fold2/pMCI/{}.nii'.format(i))
                  

033_S_0906_I27135.nii
0.7668283
036_S_0976_I35548.nii
0.38121858
127_S_1427_I73409.nii
0.445337
126_S_1077_I35544.nii
0.2814825
013_S_1186_I42880.nii
0.20424967
126_S_0865_I30553.nii
0.2471242
003_S_1074_I171424.nii
0.09887451
013_S_0860_I30124.nii
0.7715776
098_S_0160_I12406.nii
0.8277334
031_S_0294_I26400.nii
0.14438924
041_S_0549_I20366.nii
0.51182085
127_S_0394_I18767.nii
0.93824935
018_S_0142_I11262.nii
0.72880447
027_S_0408_I26953.nii
0.958254
033_S_0513_I19372.nii
0.5517367
130_S_0285_I225718.nii
0.12226255
041_S_1425_I72658.nii
0.42716476
136_S_0695_I27569.nii
0.6735573
033_S_0567_I17165.nii
0.78910565
041_S_1010_I45280.nii
0.78867674
073_S_0909_I32583.nii
0.67942053


In [16]:
dire="/local_mount/space/celer/1/users/notebooks/moises/pdata/newdata/kfold2/fold2/sMCI"

In [17]:
for i in os.listdir(dire):
                volume=process(os.path.join(dire,'{}'.format(i)))
                prediction=model.predict(volume)
                score=prediction[0][0]
                print(i)
                print(score)
                model.layers[-1].activation = None
                heatmap = make_gradcam_heatmap(volume, model, last_conv_layer_name)
                resized_heatmap = get_resized_heatmap(heatmap, volume_size)
                volume=np.squeeze(volume)
                
                input_img= nibabel.load(os.path.join(dire,'{}'.format(i)))
                aff=input_img.affine
                
                saved_img = nibabel.Nifti1Image(volume, aff)
                nibabel.save(saved_img, '/local_mount/space/celer/1/users/notebooks/moises/processed_nifty/fold2/sMCI/{}.nii'.format(i))
                
                heaty=normalize(resized_heatmap)
                saved_heatmap = nibabel.Nifti1Image(heaty, aff)
                nibabel.save(saved_heatmap, '/local_mount/space/celer/1/users/notebooks/moises/heatmaps/fold2/sMCI/{}.nii'.format(i))
                
                

109_S_6221_I1122085.nii
0.23047967
341_S_6605_I1080531.nii
0.31895316
341_S_6686_I1147777.nii
0.43177038
003_S_6432_I1024803.nii
0.18685415
019_S_6483_I1029236.nii
0.15909287
036_S_0673_I20471.nii
0.52895355
032_S_6804_I1255537.nii
0.17122094
027_S_0307_I256050.nii
0.026979461
029_S_6798_I1269654.nii
0.06687337
037_S_0377_I217256.nii
0.024987144
019_S_6757_I1185225.nii
0.0084047755
036_S_0656_I19783.nii
0.13555457
053_S_6598_I1063018.nii
0.020969251
024_S_6846_I1283729.nii
0.026074946
141_S_1052_I286096.nii
0.21978535
037_S_6141_I985518.nii
0.0061046695
109_S_6364_I1122188.nii
0.31364822
305_S_6877_I1344496.nii
0.0614008
053_S_0919_I267656.nii
0.07982048
007_S_0414_I17314.nii
0.67155445
141_S_1255_I227044.nii
0.75619847
013_S_1275_I44970.nii
0.4609256
019_S_6533_I1045673.nii
0.009448028
067_S_6525_I1183348.nii
0.031270392
129_S_6830_I1257732.nii
0.06367004
168_S_6634_I1136677.nii
0.35417083
073_S_0746_I43238.nii
0.23129243
305_S_6378_I1043319.nii
0.11855085
035_S_6947_I1462465.nii
0.24

# Loop fold 3

In [18]:
modeldir="/local_mount/space/celer/1/users/notebooks/moises/saved_models/newconv_originalsmallcnn2/epoch17"

In [19]:
model=tf.keras.models.load_model(modeldir)

In [20]:
dire="/local_mount/space/celer/1/users/notebooks/moises/pdata/newdata/kfold2/fold3/pMCI"

In [21]:
for i in os.listdir(dire):
                volume=process(os.path.join(dire,'{}'.format(i)))
                prediction=model.predict(volume)
                score=prediction[0][0]
                print(i)
                print(score)
                model.layers[-1].activation = None
                heatmap = make_gradcam_heatmap(volume, model, last_conv_layer_name)
                resized_heatmap = get_resized_heatmap(heatmap, volume_size)
                volume=np.squeeze(volume)
                
                input_img= nibabel.load(os.path.join(dire,'{}'.format(i)))
                aff=input_img.affine
                
                saved_img = nibabel.Nifti1Image(volume, aff)
                nibabel.save(saved_img, '/local_mount/space/celer/1/users/notebooks/moises/processed_nifty/fold3/pMCI/{}.nii'.format(i))
                
                heaty=normalize(resized_heatmap)
                saved_heatmap = nibabel.Nifti1Image(heaty, aff)
                nibabel.save(saved_heatmap, '/local_mount/space/celer/1/users/notebooks/moises/heatmaps/fold3/pMCI/{}.nii'.format(i))
                
                
                
                

041_S_1423_I73983.nii
0.7668433
005_S_1224_I40612.nii
0.36469913
100_S_1226_I221694.nii
0.115816355
036_S_1135_I36214.nii
0.51843166
035_S_0292_I14203.nii
0.45513046
094_S_1398_I56520.nii
0.7346072
057_S_0941_I27433.nii
0.9899648
041_S_1412_I70184.nii
0.21546856
100_S_0995_I54611.nii
0.31463513
033_S_1116_I205053.nii
0.77015454
013_S_0240_I28950.nii
0.34441552
127_S_0112_I15262.nii
0.27720395
137_S_0994_I32055.nii
0.86722577
016_S_1117_I183352.nii
0.6196538
027_S_0835_I266911.nii
0.33592016
016_S_1326_I312617.nii
0.36849743
005_S_0572_I211079.nii
0.7689615
014_S_0563_I196086.nii
0.9322581
037_S_1078_I209206.nii
0.8319696
027_S_0256_I26950.nii
0.87456465
011_S_0326_I13808.nii
0.46006116


In [22]:
dire="/local_mount/space/celer/1/users/notebooks/moises/pdata/newdata/kfold2/fold3/sMCI"

In [23]:
for i in os.listdir(dire):
                volume=process(os.path.join(dire,'{}'.format(i)))
                prediction=model.predict(volume)
                score=prediction[0][0]
                print(i)
                print(score)
                model.layers[-1].activation = None
                heatmap = make_gradcam_heatmap(volume, model, last_conv_layer_name)
                resized_heatmap = get_resized_heatmap(heatmap, volume_size)
                volume=np.squeeze(volume)
                
                input_img= nibabel.load(os.path.join(dire,'{}'.format(i)))
                aff=input_img.affine
                
                saved_img = nibabel.Nifti1Image(volume, aff)
                nibabel.save(saved_img, '/local_mount/space/celer/1/users/notebooks/moises/processed_nifty/fold3/sMCI/{}.nii'.format(i))
                
                heaty=normalize(resized_heatmap)
                saved_heatmap = nibabel.Nifti1Image(heaty, aff)
                nibabel.save(saved_heatmap, '/local_mount/space/celer/1/users/notebooks/moises/heatmaps/fold3/sMCI/{}.nii'.format(i))
                
                
                
                

141_S_1378_I55188.nii
0.17670073
941_S_6803_I1238836.nii
0.81347847
027_S_6463_I1062326.nii
0.27387512
168_S_6875_I1364446.nii
0.02510739
126_S_1187_I284312.nii
0.005714427
022_S_0544_I17557.nii
0.5493474
020_S_6901_I1420623.nii
0.0023028213
141_S_6075_I921120.nii
0.019667981
002_S_6652_I1158636.nii
0.14476435
114_S_1103_I34083.nii
0.18862955
137_S_6693_I1267099.nii
0.28857374
035_S_6480_I1024012.nii
0.008379075
116_S_6550_I1038365.nii
0.010559493
016_S_6800_I1241315.nii
0.44992763
018_S_6414_I1043696.nii
0.09600715
305_S_6899_I1415873.nii
0.021224534
130_S_6823_I1276464.nii
0.03952216
013_S_1120_I31383.nii
0.8272651
141_S_6779_I1223141.nii
0.013102686
114_S_0410_I27453.nii
0.0051089884
137_S_6654_I1160601.nii
0.0031126714
006_S_6674_I1165048.nii
0.038582977
016_S_6926_I1480980.nii
0.09346678
137_S_0158_I12120.nii
0.29980427
003_S_0908_I223689.nii
0.08442213
006_S_6770_I1225564.nii
0.05856304
003_S_1122_I222882.nii
0.21470767
126_S_0709_I20549.nii
0.6144651
114_S_0378_I29035.nii
0.3020

# Loop fold 4

In [24]:
modeldir="/local_mount/space/celer/1/users/notebooks/moises/saved_models/newconv_originalsmallcnn3/epoch22"

In [25]:
model=tf.keras.models.load_model(modeldir)

In [26]:
dire="/local_mount/space/celer/1/users/notebooks/moises/pdata/newdata/kfold2/fold4/pMCI"

In [27]:
for i in os.listdir(dire):
                volume=process(os.path.join(dire,'{}'.format(i)))
                prediction=model.predict(volume)
                score=prediction[0][0]
                print(i)
                print(score)
                model.layers[-1].activation = None
                heatmap = make_gradcam_heatmap(volume, model, last_conv_layer_name)
                resized_heatmap = get_resized_heatmap(heatmap, volume_size)
                volume=np.squeeze(volume)
                
                input_img= nibabel.load(os.path.join(dire,'{}'.format(i)))
                aff=input_img.affine
                
                saved_img = nibabel.Nifti1Image(volume, aff)
                nibabel.save(saved_img, '/local_mount/space/celer/1/users/notebooks/moises/processed_nifty/fold4/pMCI/{}.nii'.format(i))
                
                heaty=normalize(resized_heatmap)
                saved_heatmap = nibabel.Nifti1Image(heaty, aff)
                nibabel.save(saved_heatmap, '/local_mount/space/celer/1/users/notebooks/moises/heatmaps/fold4/pMCI/{}.nii'.format(i))
                
                
                
                

035_S_0997_I36328.nii
0.60192835
114_S_1106_I33508.nii
0.021413952
011_S_1282_I44227.nii
0.32485533
016_S_0702_I193103.nii
0.27333498
011_S_0861_I26521.nii
0.274518
037_S_0566_I260904.nii
0.9807632
941_S_1311_I76467.nii
0.29354954
126_S_0708_I23475.nii
0.5856493
098_S_0667_I193491.nii
0.87101346
100_S_0930_I53036.nii
0.9210909
007_S_0101_I9904.nii
0.2280957
130_S_0289_I228746.nii
0.6151555
127_S_1032_I32437.nii
0.37923288
116_S_0361_I15903.nii
0.32844046
941_S_1295_I69881.nii
0.5598418
052_S_0671_I243795.nii
0.9888299
098_S_0269_I241729.nii
0.98113054
007_S_0128_I20483.nii
0.81711876
057_S_1007_I29480.nii
0.91720235
002_S_1268_I228411.nii
0.6728559
128_S_1043_I272435.nii
0.9507189


In [28]:
dire="/local_mount/space/celer/1/users/notebooks/moises/pdata/newdata/kfold2/fold4/sMCI"

In [29]:
for i in os.listdir(dire):
                volume=process(os.path.join(dire,'{}'.format(i)))
                prediction=model.predict(volume)
                score=prediction[0][0]
                print(i)
                print(score)
                model.layers[-1].activation = None
                heatmap = make_gradcam_heatmap(volume, model, last_conv_layer_name)
                resized_heatmap = get_resized_heatmap(heatmap, volume_size)
                volume=np.squeeze(volume)
                
                input_img= nibabel.load(os.path.join(dire,'{}'.format(i)))
                aff=input_img.affine
                
                saved_img = nibabel.Nifti1Image(volume, aff)
                nibabel.save(saved_img, '/local_mount/space/celer/1/users/notebooks/moises/processed_nifty/fold4/sMCI/{}.nii'.format(i))
                
                heaty=normalize(resized_heatmap)
                saved_heatmap = nibabel.Nifti1Image(heaty, aff)
                nibabel.save(saved_heatmap, '/local_mount/space/celer/1/users/notebooks/moises/heatmaps/fold4/sMCI/{}.nii'.format(i))
                
                
                

109_S_1114_I35600.nii
0.46172842
005_S_6427_I1029563.nii
0.02403716
098_S_6707_I1192255.nii
0.048404355
135_S_6446_I1019812.nii
0.028327052
027_S_0644_I242306.nii
0.17217138
153_S_6450_I1027402.nii
0.39801094
024_S_1400_I55794.nii
0.15960515
016_S_6809_I1237158.nii
0.21580936
068_S_0802_I916314.nii
0.14621766
131_S_6805_I1268415.nii
0.751105
003_S_6954_I1499518.nii
0.067999415
041_S_6731_I1190810.nii
0.10537068
129_S_6857_I1316777.nii
0.2821398
127_S_1419_I71273.nii
0.36581492
022_S_6847_I1287387.nii
0.04732389
032_S_6700_I1242413.nii
0.90073943
003_S_6258_I996084.nii
0.02091969
041_S_1418_I59451.nii
0.21848185
070_S_6911_I1438641.nii
0.31444108
126_S_6724_I1178514.nii
0.48006755
036_S_6897_I1450142.nii
0.0445129
006_S_6610_I1075932.nii
0.053195205
128_S_0200_I1080246.nii
0.3797613
019_S_6635_I1122508.nii
0.11418857
016_S_6771_I1260305.nii
0.6223705
006_S_6657_I1140902.nii
0.12294582
141_S_6964_I1480309.nii
0.0046682125
168_S_6426_I1023676.nii
0.0037871087
006_S_6677_I1228527.nii
0.247

# Loop fold 5

In [30]:
modeldir="/local_mount/space/celer/1/users/notebooks/moises/saved_models/newconv_originalsmallcnn4/epoch9"

In [31]:
model=tf.keras.models.load_model(modeldir)

In [32]:
dire="/local_mount/space/celer/1/users/notebooks/moises/pdata/newdata/kfold2/fold5/pMCI"

In [33]:
for i in os.listdir(dire):
                volume=process(os.path.join(dire,'{}'.format(i)))
                prediction=model.predict(volume)
                score=prediction[0][0]
                print(i)
                print(score)
                model.layers[-1].activation = None
                heatmap = make_gradcam_heatmap(volume, model, last_conv_layer_name)
                resized_heatmap = get_resized_heatmap(heatmap, volume_size)
                volume=np.squeeze(volume)
                
                input_img= nibabel.load(os.path.join(dire,'{}'.format(i)))
                aff=input_img.affine
                
                saved_img = nibabel.Nifti1Image(volume, aff)
                nibabel.save(saved_img, '/local_mount/space/celer/1/users/notebooks/moises/processed_nifty/fold5/pMCI/{}.nii'.format(i))
                
                heaty=normalize(resized_heatmap)
                saved_heatmap = nibabel.Nifti1Image(heaty, aff)
                nibabel.save(saved_heatmap, '/local_mount/space/celer/1/users/notebooks/moises/heatmaps/fold5/pMCI/{}.nii'.format(i))
                
                
                

012_S_1033_I31433.nii
0.3295935
037_S_0552_I240801.nii
0.9906549
014_S_0658_I202817.nii
0.36944807
021_S_0141_I20254.nii
0.6194032
003_S_1057_I223690.nii
0.9469779
018_S_0080_I11044.nii
0.3960523
100_S_0892_I51323.nii
0.6974042
018_S_0057_I11042.nii
0.7855923
002_S_0729_I187916.nii
0.18854006
128_S_0227_I312075.nii
0.96967095
036_S_1240_I44117.nii
0.93051857
021_S_0626_I20198.nii
0.112107694
127_S_0925_I29343.nii
0.7846106
116_S_0834_I201222.nii
0.95272887
041_S_0314_I14488.nii
0.51543504
024_S_1393_I49954.nii
0.8840489
031_S_0830_I194250.nii
0.9611423
032_S_0214_I12440.nii
0.6175738
036_S_0945_I29098.nii
0.40472734
029_S_1318_I141742.nii
0.047322094
033_S_0511_I17296.nii
0.7947234
068_S_0872_I282989.nii
0.035844326


In [34]:
dire="/local_mount/space/celer/1/users/notebooks/moises/pdata/newdata/kfold2/fold5/sMCI"

In [35]:
for i in os.listdir(dire):
                volume=process(os.path.join(dire,'{}'.format(i)))
                prediction=model.predict(volume)
                score=prediction[0][0]
                print(i)
                print(score)
                model.layers[-1].activation = None
                heatmap = make_gradcam_heatmap(volume, model, last_conv_layer_name)
                resized_heatmap = get_resized_heatmap(heatmap, volume_size)
                volume=np.squeeze(volume)
                
                input_img= nibabel.load(os.path.join(dire,'{}'.format(i)))
                aff=input_img.affine
                
                saved_img = nibabel.Nifti1Image(volume, aff)
                nibabel.save(saved_img, '/local_mount/space/celer/1/users/notebooks/moises/processed_nifty/fold5/sMCI/{}.nii'.format(i))
                
                heaty=normalize(resized_heatmap)
                saved_heatmap = nibabel.Nifti1Image(heaty, aff)
                nibabel.save(saved_heatmap, '/local_mount/space/celer/1/users/notebooks/moises/heatmaps/fold5/sMCI/{}.nii'.format(i))
                
                
                
                

067_S_6529_I1059604.nii
0.61401665
012_S_6073_I916397.nii
0.3273858
305_S_6845_I1286989.nii
0.16618063
131_S_6143_I1013164.nii
0.78265244
153_S_6274_I984660.nii
0.18638296
037_S_6627_I1136170.nii
0.23576136
011_S_0362_I14183.nii
0.70399964
022_S_6716_I1214884.nii
0.0045251483
051_S_6761_I1188820.nii
0.12898216
012_S_0634_I20175.nii
0.29849926
094_S_1188_I38073.nii
0.0740318
094_S_0531_I20285.nii
0.42139402
006_S_6696_I1181907.nii
0.37809914
041_S_0282_I15580.nii
0.11440096
027_S_6842_I1299327.nii
0.057589877
002_S_6695_I1152976.nii
0.8660385
094_S_1417_I65420.nii
0.19653675
036_S_6885_I1343472.nii
0.381462
067_S_6474_I1037264.nii
0.02327233
006_S_6252_I995001.nii
0.62911654
016_S_0354_I17384.nii
0.06691415
137_S_0668_I194243.nii
0.025395257
012_S_1165_I36389.nii
0.35490617
099_S_6691_I1380833.nii
0.59098184
135_S_6586_I1063261.nii
0.23929688
137_S_0800_I23686.nii
0.18639688
127_S_1210_I41570.nii
0.90565884
305_S_6871_I1321238.nii
0.043958653
022_S_6863_I1333377.nii
0.39363214
036_S_691